In [1]:
import os
from pathlib import Path
import torch
import re
import random
import torch
from torch.utils.data import Dataset
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordPieceTrainer
import random
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import tqdm
from torch.utils.data import Dataset, DataLoader
import itertools
import math
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam

In [2]:
merge= r'/kaggle/input/wikidata/diamond_merged.txt' # Parimelagar urai with its explanation and  Random Merge 20 times
# train = r'../datasets/main_train_train.txt'
test = r'/kaggle/input/wikidata/main_train_valid.txt' # total 400MB
iconst=r'/kaggle/input/iconst/data.txt'

In [3]:
conv=[]
# with open( merge, 'r',encoding='utf-8') as c:
#     merged = c.readlines()           
# # with open( train, 'r',encoding='utf-8') as c:
# #     trained = c.readlines()
# with open( test, 'r',encoding='utf-8') as c:
#     tested = c.readlines()
with open( iconst, 'r',encoding='utf-8') as c:
    icst = c.readlines()  
conv=icst

In [27]:
# conv=tested+merged


In [4]:
len(conv)#,len(trained),len(tested) Total  sentences

6396

In [5]:
sum=0
l=0
dic={}
word_freq_count={}
s=set()
for sent in conv:
    length=len(sent.split())
    l=max(l,length)
    for word in sent.split():
        if word in word_freq_count.keys():
            word_freq_count[word]=word_freq_count[word]+1
        else:
            word_freq_count[word]=1
        s.add(word)
    dic[length]=1 if length not in dic.keys() else dic[length]+1
len(s),l                                                           

(22324, 233)

In [6]:
word_freq_count_above_10={}
for key,value in word_freq_count.items():
    if value>10:
        word_freq_count_above_10[key]=value
print(len(word_freq_count_above_10))

1769


In [7]:
sorted_dict = dict(sorted(dic.items()))
sorted_dict    #sentence Length

{0: 2,
 1: 24,
 2: 72,
 3: 93,
 4: 109,
 5: 122,
 6: 128,
 7: 142,
 8: 142,
 9: 141,
 10: 129,
 11: 129,
 12: 135,
 13: 141,
 14: 126,
 15: 141,
 16: 130,
 17: 136,
 18: 151,
 19: 152,
 20: 148,
 21: 109,
 22: 142,
 23: 128,
 24: 134,
 25: 122,
 26: 118,
 27: 133,
 28: 126,
 29: 84,
 30: 112,
 31: 116,
 32: 102,
 33: 94,
 34: 122,
 35: 98,
 36: 105,
 37: 93,
 38: 120,
 39: 106,
 40: 122,
 41: 112,
 42: 150,
 43: 164,
 44: 145,
 45: 174,
 46: 161,
 47: 140,
 48: 105,
 49: 93,
 50: 95,
 51: 55,
 52: 41,
 53: 36,
 54: 27,
 55: 14,
 56: 12,
 57: 12,
 58: 7,
 59: 7,
 60: 5,
 61: 3,
 62: 3,
 63: 1,
 65: 3,
 66: 1,
 67: 2,
 69: 3,
 70: 2,
 71: 4,
 72: 3,
 73: 1,
 75: 2,
 77: 1,
 78: 1,
 80: 1,
 81: 1,
 82: 2,
 83: 1,
 85: 1,
 87: 2,
 90: 1,
 92: 1,
 94: 1,
 96: 1,
 97: 1,
 99: 2,
 104: 1,
 105: 1,
 107: 1,
 109: 1,
 114: 1,
 116: 1,
 118: 1,
 122: 1,
 131: 1,
 132: 1,
 151: 1,
 156: 1,
 159: 1,
 161: 1,
 165: 1,
 180: 1,
 233: 1}

process labels

In [9]:
import json

with open(r'/kaggle/input/iconst/labels.json', 'r') as f:
    data_v = json.load(f)

labels = [data_v[key] for key in sorted(data_v.keys())]

In [10]:
labels[:5]

[['0', '0', '0', '0'],
 ['1', '1', '2', '3'],
 ['1', '0', '1', '1'],
 ['1', '1', '2', '1'],
 ['1', '0', '1', '1']]

In [36]:
# import string
# def remove_punctuation(sentences):
#     translator = str.maketrans('', '', string.punctuation)
#     cleaned_sentences = []
#     for sentence in sentences:
#         words = sentence.translate(translator).split()
#         cleaned_sentence = " ".join(words)
#         cleaned_sentences.append(cleaned_sentence)
    
#     return cleaned_sentences

In [37]:
# # # conv=trained
# train_cleaned=remove_punctuation(trained)
# test_cleaned=remove_punctuation(tested)

In [38]:
# with open('main_train_train.txt','a',encoding='utf-8') as f:
#     for line in train_cleaned:
#         if len(line.split())<=64: 
#             f.writelines(f'{line}\n')
# with open('main_train_valid.txt','a',encoding='utf-8') as f:
#     for line in test_cleaned:
#         if len(line.split())<=64: 
#             f.writelines(f'{line}\n')
    

In [26]:
conv[:5]

['USER Abhi tak 2000 ke note me mujhe GPS nano chip nahin mila\n',
 'USER USER Abe katiye tumse kuch huaa toh jata nahi bas bakri jaise maymay karte rehte ho behan ke lodo tumhari Tablighi jamaat ke karan corona faila aur Bhoomi Poojan mein sab ko screen aur full medical jach ke saath huaa bhosdiwalo\n',
 'USER Ye sab sazish hai bina saman ke koi kaise apne gher ja sakta hai dekh lena inn logo ke beech bahut aise jamaat ke wo corona honge jo delhi se nikle they\n',
 'abe jao tum to dasko pahle hi fash gye the jab tere dada ne talwar ke nokh par salwar pahena tha tera daram kabhi hindu kabhi muslim\n',
 'Ab ye afbah kaun faila Raha hai ki Shahhen bag ke Biryani me Population control ki Dawa Milli Hui hai\n']

In [40]:
# conv=trained
# print(len(conv),len(trained))

In [11]:
# paths=['/kaggle/input/wikidata/diamond_merged.txt','/kaggle/input/wikidata/main_train_valid.txt' ]
paths=['/kaggle/input/iconst/data.txt']

In [75]:
import torch
from torch.utils.data import Dataset, DataLoader
from tokenizers import Tokenizer, models, pre_tokenizers, processors
import random
import math
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from tokenizers import trainers
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors
import torch
from torch.utils.data import Dataset
import random
import json
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

SEQ_LEN = 68
EMBED_LEN = 252
VOCAB_SIZE = 20000   #vocab size

def create_tokenizer(paths, vocab_size=VOCAB_SIZE):
    tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    special_tokens = ["[CLS1]","[CLS2]","[PAD]", "[MASK]", "[UNK]"]
    tokenizer.add_special_tokens(special_tokens)

    def batch_iterator(file_paths, batch_size=64):
        for path in file_paths:
            with open(path, 'r', encoding='utf-8') as f:
                for line in f:
                    yield line.strip()

    trainer = WordPieceTrainer(
        vocab_size=vocab_size,
        special_tokens=special_tokens
    )
    tokenizer.train_from_iterator(batch_iterator(paths), trainer=trainer)
    
    return tokenizer

class BERTDataset(Dataset):
    def __init__(self, sentences, tokenizer, labels,seq_len=SEQ_LEN):
        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.sentences = sentences
        self.labels=labels

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, item,flag=False):
        sentence = self.sentences[item]
        encoding = self.tokenizer.encode(sentence)
        
        # List of special class tokens
        classes = [
            self.tokenizer.token_to_id('[CLS1]'),
            self.tokenizer.token_to_id('[CLS2]'),
            # self.tokenizer.token_to_id('[CLS3]'),
            # self.tokenizer.token_to_id('[CLS4]')
        ]
        
        tokens = classes + encoding.ids  
        if len(tokens) > self.seq_len:
            tokens = tokens[:self.seq_len]
        
        padding_length = self.seq_len - len(tokens)
        tokens = tokens + [self.tokenizer.token_to_id('[PAD]')] * padding_length
        
        mask = [1 if token != self.tokenizer.token_to_id('[PAD]') else 0 for token in tokens]
        
        bert_input = tokens.copy()
        bert_label = [int(self.labels[item][0]), int(self.labels[item][1]), int(self.labels[item][2]), int(self.labels[item][3])]
    
        if flag:
            for i in range(len(tokens)):
                if mask[i] == 1 and random.random() < 0.25:
                    if random.random() < 0.8:
                        bert_input[i] = self.tokenizer.token_to_id('[MASK]')
                    elif random.random() < 0.5:
                        bert_input[i] = random.randint(0, self.tokenizer.get_vocab_size() - 1)
        
        return {
            'bert_input': torch.tensor(bert_input),
            'bert_label': torch.tensor(bert_label),
            'attention_mask': torch.tensor(mask)
        }

    
class PositionalEmbedding(torch.nn.Module):
    def __init__(self, d_model, max_len=SEQ_LEN):
        super().__init__()
        pe = torch.zeros(max_len, d_model).float()
        print(pe.shape)
        pe.require_grad = False
        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        print('div_term',div_term)
        print('div_term_dim',div_term.shape)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term) 
        print('pe s',pe.shape)
        print('pe',pe)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return self.pe[:, :x.size(1)]

class BERTEmbedding(torch.nn.Module):
    def __init__(self, vocab_size, embed_size, seq_len=SEQ_LEN, dropout=0.1):
        super().__init__()
        self.token = torch.nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.position = PositionalEmbedding(embed_size, seq_len)
        self.dropout = torch.nn.Dropout(p=dropout)
        self.embed_size = embed_size

    def forward(self, sequence):
        x = self.token(sequence) + self.position(sequence)
        return self.dropout(x)

class MultiHeadedAttention(torch.nn.Module):
    def __init__(self, heads, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % heads == 0
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = torch.nn.Dropout(dropout)
        self.query = torch.nn.Linear(d_model, d_model)
        self.key = torch.nn.Linear(d_model, d_model)
        self.value = torch.nn.Linear(d_model, d_model)
        self.output_linear = torch.nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask):
        batch_size = query.size(0)
        query = self.query(query).view(batch_size, -1, self.heads, self.d_k).transpose(1, 2)
        key = self.key(key).view(batch_size, -1, self.heads, self.d_k).transpose(1, 2)
        value = self.value(value).view(batch_size, -1, self.heads, self.d_k).transpose(1, 2)

        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.size(-1))
        scores = scores.masked_fill(mask == 0, -1e9)

        weights = F.softmax(scores, dim=-1)
        weights = self.dropout(weights)

        context = torch.matmul(weights, value)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.heads * self.d_k)
        return self.output_linear(context)

class BERT(torch.nn.Module):
    def __init__(self, vocab_size=VOCAB_SIZE, d_model=EMBED_LEN, n_layers=12, heads=12, dropout=0.1):
        super().__init__()
        self.embedding = BERTEmbedding(vocab_size, d_model, SEQ_LEN)
        self.encoder_layers = torch.nn.ModuleList([EncoderLayer(d_model, heads, d_model * 4, dropout) for _ in range(n_layers)])

    def forward(self, x, attention_mask):
        x = self.embedding(x)
        mask = attention_mask.unsqueeze(1).unsqueeze(2)
        for layer in self.encoder_layers:
            x = layer(x, mask)
        return x
        
class ClassificationModel(torch.nn.Module):
    def __init__(self, hidden):
        super().__init__()
        # Create separate linear layers for each classification task
        self.linear1 = torch.nn.Linear(hidden, 2)
        self.linear2 = torch.nn.Linear(hidden, 2)
        self.softmax = torch.nn.LogSoftmax(dim=-1)
    
    def forward(self, x,task):
        # Extract tokens
        if(task==1):
            token=x[:, 0, :]
        else:
            token=x[:, 1, :]
        # Apply linear layers and find the class indices directly
        class_indices = torch.stack([
            torch.argmax(self.linear1(token1), dim=-1),  # Shape: (batch_size,)
            torch.argmax(self.linear2(token2), dim=-1),  # Shape: (batch_size,)
            torch.argmax(self.linear3(token3), dim=-1),  # Shape: (batch_size,)
            torch.argmax(self.linear4(token4), dim=-1)   # Shape: (batch_size,)
        ])  # Shape: (4, batch_size)

        # Transpose to get the desired shape (batch_size, 4)
        class_indices = class_indices.transpose(0, 1)  # Shape: (batch_size, 4)

        return class_indices
        
class BERTLM(torch.nn.Module):
    def __init__(self, bert: BERT, vocab_size):
        super().__init__()
        self.bert = bert
        self.class_lm=ClassificationModel(self.bert.embedding.embed_size)

    def forward(self, x, attention_mask,task):
        x = self.bert(x, attention_mask)
        return self.class_lm(x,task)

class FeedForward(torch.nn.Module):
    def __init__(self, d_model, middle_dim=2048, dropout=0.1):
        super(FeedForward, self).__init__()
        self.fc1 = torch.nn.Linear(d_model, middle_dim)
        self.fc2 = torch.nn.Linear(middle_dim, d_model)
        self.dropout = torch.nn.Dropout(dropout)
        self.activation = torch.nn.GELU()

    def forward(self, x):
        out = self.activation(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out

class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model=EMBED_LEN, heads=12, feed_forward_hidden=EMBED_LEN*4, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.layernorm = torch.nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadedAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model, middle_dim=feed_forward_hidden)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, embeddings, mask):
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        interacted = self.layernorm(interacted + embeddings)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded

def train_bert(model, train_dataloader, optimizer,scheduler, device, epoch_start,epoch_end):
    model.train()
    for epoch in range(epoch_start,epoch_end):
        total_loss = 0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
            optimizer.zero_grad()
            bert_input = batch['bert_input'].to(device)
            bert_label = batch['bert_label'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            print(bert_input.shape)
            outputs = model(bert_input, attention_mask)
            
            print('outputs',outputs)
            print('labels',bertlabels)
       
            loss=find_f1_loss(outputs,bert_labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
            
            
        if((epoch+1)%2==0):
            save_directory = f"/kaggle/working/epoch{epoch}"
            save_model(model, tokenizer, save_directory)

        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")


def find_similar_words(word, tokenizer, model, top_n=5):
    model.eval()
    with torch.no_grad():
        word_id = tokenizer.token_to_id(word)
        if word_id is None:
            return []
        word_embedding = model.bert.embedding.token.weight[word_id]
        cosine_similarities = F.cosine_similarity(word_embedding, model.bert.embedding.token.weight)
        top_indices = torch.argsort(cosine_similarities, descending=True)[1:top_n+1]
        print(top_indices)
        return [tokenizer.id_to_token(idx.item()) for idx in top_indices]

def print_vocabulary(tokenizer):
    vocab = tokenizer.get_vocab()
    sorted_vocab = sorted(vocab.items(), key=lambda x: x[1])
    print("Vocabulary:")
    for word, index in sorted_vocab:
        print(f"{index}: {word}")
    print(f"Total vocabulary size: {len(vocab)}")


In [76]:
# tamil_text_files = ['/kaggle/input/wikidata/diamond_merged.txt','/kaggle/input/wikidata/main_train_valid.txt' ]
tamil_text_files=['/kaggle/input/iconst/data.txt']
tokenizer = create_tokenizer(tamil_text_files)  

In [77]:
dataset = BERTDataset(conv, tokenizer,labels)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

print(f"Vocabulary size: {tokenizer.get_vocab_size()}")
print(f"Number of sentences: {len(dataset)}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

bert = BERT(vocab_size=VOCAB_SIZE, d_model=EMBED_LEN, n_layers=12, heads=12)
model = BERTLM(bert, vocab_size=VOCAB_SIZE)
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
WARMUP_STEPS=10000
EPOCHS=20
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=total_steps)

Vocabulary size: 20000
Number of sentences: 6396
Using device: cuda
torch.Size([68, 252])
div_term tensor([1.0000e+00, 9.2951e-01, 8.6399e-01, 8.0309e-01, 7.4648e-01, 6.9386e-01,
        6.4495e-01, 5.9948e-01, 5.5723e-01, 5.1795e-01, 4.8144e-01, 4.4750e-01,
        4.1596e-01, 3.8664e-01, 3.5938e-01, 3.3405e-01, 3.1050e-01, 2.8861e-01,
        2.6827e-01, 2.4936e-01, 2.3178e-01, 2.1544e-01, 2.0026e-01, 1.8614e-01,
        1.7302e-01, 1.6082e-01, 1.4949e-01, 1.3895e-01, 1.2915e-01, 1.2005e-01,
        1.1159e-01, 1.0372e-01, 9.6411e-02, 8.9615e-02, 8.3298e-02, 7.7426e-02,
        7.1969e-02, 6.6895e-02, 6.2180e-02, 5.7797e-02, 5.3723e-02, 4.9936e-02,
        4.6416e-02, 4.3144e-02, 4.0103e-02, 3.7276e-02, 3.4648e-02, 3.2206e-02,
        2.9936e-02, 2.7826e-02, 2.5864e-02, 2.4041e-02, 2.2346e-02, 2.0771e-02,
        1.9307e-02, 1.7946e-02, 1.6681e-02, 1.5505e-02, 1.4412e-02, 1.3396e-02,
        1.2452e-02, 1.1574e-02, 1.0758e-02, 1.0000e-02, 9.2951e-03, 8.6399e-03,
        8.0309e-03, 7

In [78]:
train_bert(model,train_loader,optimizer,scheduler,device,0,1) 

Epoch 1:   0%|          | 0/200 [00:00<?, ?it/s]

torch.Size([32, 68])


Epoch 1:   0%|          | 0/200 [00:00<?, ?it/s]

outputs tensor([[1, 1, 3, 1],
        [1, 1, 3, 0],
        [1, 1, 3, 1],
        [1, 1, 0, 0],
        [1, 1, 0, 1],
        [0, 1, 0, 1],
        [0, 1, 3, 1],
        [1, 1, 3, 1],
        [1, 1, 3, 2],
        [1, 1, 0, 1],
        [1, 1, 0, 1],
        [1, 1, 3, 1],
        [1, 1, 3, 1],
        [1, 1, 3, 2],
        [1, 1, 0, 1],
        [1, 1, 3, 1],
        [1, 1, 0, 0],
        [1, 1, 3, 2],
        [0, 1, 0, 1],
        [1, 1, 3, 1],
        [1, 1, 3, 1],
        [1, 1, 0, 0],
        [0, 1, 1, 1],
        [0, 1, 3, 2],
        [1, 1, 3, 1],
        [0, 1, 0, 1],
        [1, 1, 3, 1],
        [1, 1, 3, 1],
        [0, 1, 3, 1],
        [1, 1, 0, 1],
        [1, 1, 3, 1],
        [1, 1, 3, 1]], device='cuda:0')
labels [['0', '0', '0', '0'], ['1', '1', '2', '3'], ['1', '0', '1', '1'], ['1', '1', '2', '1'], ['1', '0', '1', '1'], ['0', '0', '0', '0'], ['0', '0', '0', '0'], ['1', '1', '2', '1'], ['0', '0', '0', '0'], ['1', '0', '2', '2'], ['1', '1', '2', '2'], ['0', '1', '0', '0']

NameError: name 'find_f1_loss' is not defined

def load_model(load_directory,model):
    model_path = os.path.join(load_directory, "bert_model_17_epoch.pth")
    print(f"Model and vocabulary loaded from {load_directory}")
    model.load_state_dict(torch.load(model_path))
    return model
model=load_model("/kaggle/input/bert/pytorch/abc/1", model)

In [57]:
# def load_model(load_directory,model): 
#     model_path = os.path.join(load_directory, "bert_model_24_epoch.pth") 
#     print(f"Model and vocabulary loaded from {load_directory}") 
#     model.load_state_dict(torch.load(model_path)) 
#     return model 
# model=load_model("/kaggle/input/abcd/pytorch/default/1/", model)

In [13]:
model.bert

BERT(
  (embedding): BERTEmbedding(
    (token): Embedding(50000, 252, padding_idx=0)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layers): ModuleList(
    (0-11): 12 x EncoderLayer(
      (layernorm): LayerNorm((252,), eps=1e-05, elementwise_affine=True)
      (self_multihead): MultiHeadedAttention(
        (dropout): Dropout(p=0.1, inplace=False)
        (query): Linear(in_features=252, out_features=252, bias=True)
        (key): Linear(in_features=252, out_features=252, bias=True)
        (value): Linear(in_features=252, out_features=252, bias=True)
        (output_linear): Linear(in_features=252, out_features=252, bias=True)
      )
      (feed_forward): FeedForward(
        (fc1): Linear(in_features=252, out_features=1008, bias=True)
        (fc2): Linear(in_features=1008, out_features=252, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (activation): GELU(approximate='none')
      )
      (dropout): Dropout(

In [17]:
model.bert.embedding.position.pe.shape

torch.Size([1, 68, 252])

In [18]:
all_embedding=model.bert.embedding.token.weight
vocab = tokenizer.get_vocab()

extract embedding from bert

In [ ]:
import torch
import torch.nn.functional as F

def get_token_embeddings_custom_bert(model, input_ids):
    print(input_ids)
    with torch.no_grad():
        embeddings = model.bert.embedding(input_ids) 
        for layer in model.bert.encoder_layers:
            embeddings = layer(embeddings)  
    
    return embeddings

def get_embedding(model, tokenizer, word, top_n=10):
    dataset = BERTDataset([word], tokenizer)
    data=dataset.__getitem__(0,flag=False)
    input_ids = data['bert_input'].unsqueeze(0).to(device)
    attention_mask = data['attention_mask'].unsqueeze(0).to(device)
    masked_lm_labels = data['bert_label'].unsqueeze(0).to(device)  
    
    word_embeddings = get_token_embeddings_custom_bert(model, input_ids)
    word_embedding = torch.mean(word_embeddings, dim=1).squeeze(0)

    # Now compute cosine similarity between this word's embedding and all token embeddings
    all_embeddings = model.bert.embedding.token.weight 
    cosine_similarities = F.cosine_similarity(word_embedding.unsqueeze(0), all_embeddings, dim=1)

    # Get the top N most similar tokens (excluding the queried word itself)
    top_indices = torch.argsort(cosine_similarities, descending=True)[1:top_n+1]
    
    # Convert the token indices back to words
    similar_words = [tokenizer.convert_ids_to_tokens(idx.item()) for idx in top_indices]
    
    print(f"Top {top_n} words similar to '{word}': {similar_words}")

word = "குடியரசு"
get_embedding(model, tokenizer, word, top_n=10)


In [25]:
find_similar_words('முதல',tokenizer,model)

tensor([37540, 27365, 12787,  6861, 21004], device='cuda:0')


['செய்தக்க', 'இப்பாடசாலை', '##ளையின்', 'Al', 'நாயர்']

In [55]:
special_tokens = ["[PAD]", "[MASK]", "[UNK]"]
vocab=tokenizer.get_vocab()
len(vocab)
for i in special_tokens:
    print(vocab[i])

0
1
2


In [10]:
key_int_value_word={}
for key,value in vocab.items():
    key_int_value_word[value]=key

check the unknown count in the built tensors for the sentence input

In [64]:
# count=0
# for i in range(80000,85000):
#     example = dataset[i]
# #     print(f"Example {i + 1}:")
#     a= example['bert_input']
# #     print("BERT Label Tokens: ", example['bert_label'])
# #     print("Attention Mask: ", example['attention_mask'])
# # #     print() 
#     a=a.tolist()
# #     l=[]
#     for j in a:
# #         l.append(key_int_value_word[j])
#         if j==2:
#             count+=1
#         if j==0:
#             break
#     if i%1000==0:
#             print(i)
# #         print(j,end=' ')
# #     print()
# #     print(l)
# print(count)

80000
81000
82000
83000
84000
0


In [17]:
vocab=tokenizer.get_vocab()
# sorted_vocab = dict(sorted(vocab.items(), key=lambda item: item[1]))

# with open('output_wiki_test1.txt','w',encoding='utf-8') as f:
#     for key,value in sorted_vocab.items():
#         f.write(f'{key}\t\t{value} \n')   

In [31]:

def test_bert_model(model, tokenizer, sentence, device,target,top_n=5):
    dataset = BERTDataset([sentence], tokenizer)
    dataset_target=BERTDataset([target],tokenizer)
    data_target=dataset_target.__getitem__(0,False)
    data=dataset.__getitem__(0,False)
#     print(data)
#     print(data_target)
    target_list_tokenized=[]
    target_input_ids=data_target['bert_input'].unsqueeze(0).to(device)
    target_list=target_input_ids.tolist()
    
    for i in target_list[0]:
        if i==0:
            break
        else:
            target_list_tokenized.append(i)
    #print('target',target_list_tokenized)
    input_ids = data['bert_input'].unsqueeze(0).to(device)
    #print(input_ids)
    attention_mask = data['attention_mask'].unsqueeze(0).to(device)
    
    key_indexes=[input_ids.tolist()[0].index(i) for i in target_list_tokenized ]
    print(key_indexes)
    
    #print(input_ids.shape)
    model.eval()
    predictions=[]
    with torch.no_grad():
        outputs=model.bert(input_ids, attention_mask)
        #print('out',outputs.shape)
    
    context_embedding_list=[]
    output=outputs.tolist()
    for i in key_indexes:
        context_embedding_list.append(output[0][i])
    context_embedding=[0]*252
    for context in context_embedding_list:
        context_embedding = [context_embedding[i] + context[i] for i in range(len(context))]
   # print(len(context_embedding))
    context_embedding_tensor = torch.tensor(context_embedding, dtype=torch.float32).to(device)
    #print(context_embedding_tensor)
    all_embeddings = model.bert.embedding.token.weight 
    cosine_similarities = F.cosine_similarity(context_embedding_tensor.unsqueeze(0), all_embeddings, dim=1)

    # Get the top N most similar tokens (excluding the queried word itself)
    top_indices = torch.argsort(cosine_similarities, descending=True)[1:10]
    print('top',top_indices)
    # Convert the token indices back to words
    return [tokenizer.id_to_token(idx.item()) for idx in top_indices]
    

In [ ]:
test_sentence = "அகர முதல எழுத்தெல்லாம் ஆதி பகவன் முதற்றே உலகு"
print(test_bert_model(model, tokenizer, test_sentence, device,"அகர"))

In [33]:
import torch
import torch.nn.functional as F

def predict_masked_words(model, tokenizer, sentence, device):
    model.eval()
    tokens = tokenizer.encode(sentence).ids
    print('tokens',tokens)
    mask_token_id = tokenizer.token_to_id('[MASK]')
    print('mask token id',mask_token_id)
    mask_positions = [i for i, token in enumerate(tokens) if token == mask_token_id]
    print('mask_position',mask_positions)
    input_ids = torch.tensor([tokens]).to(device)
    print('input ids',input_ids)
    attention_mask = torch.ones_like(input_ids).to(device)
    print('att',attention_mask)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
    print('outputs',outputs.shape)
    predictions = []
    for pos in range(10):
        predicted_id = outputs[0, pos].argmax().item()
        predicted_token = tokenizer.id_to_token(predicted_id)
        predictions.append((pos, predicted_token))
    
    return predictions

# def analyze_training_performance(losses, epochs):
#     import matplotlib.pyplot as plt
    
#     plt.figure(figsize=(10, 6))
#     plt.plot(range(1, epochs+1), losses)
#     plt.title('Training Loss over Epochs')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.grid(True)
#     plt.show()
    
#     print(f"Initial loss: {losses[0]:.4f}")
#     print(f"Final loss: {losses[-1]:.4f}")
#     print(f"Absolute improvement: {losses[0] - losses[-1]:.4f}")
#     print(f"Relative improvement: {(losses[0] - losses[-1]) / losses[0] * 100:.2f}%")


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
test_sentence = "அகர [MASK] எழுத்தெல்லாம் ஆதி [MASK] முதற்றே உலகு"
predictions = predict_masked_words(model, tokenizer, test_sentence, device)

print("Original sentence:", test_sentence)
print("Predictions:")
for pos, pred in predictions:
    print(f"Position {pos}: {pred}")

# # Assuming you've collected losses during training
# training_losses = [...]  # List of loss values from each epoch
# analyze_training_performance(training_losses, len(training_losses))

tokens [12644, 1, 3780, 4426, 5901, 1, 14422, 1283, 1271, 4958]
mask token id 1
mask_position [1, 5]
input ids tensor([[12644,     1,  3780,  4426,  5901,     1, 14422,  1283,  1271,  4958]],
       device='cuda:0')
att tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
outputs torch.Size([1, 10, 50000])
Original sentence: அகர [MASK] எழுத்தெல்லாம் ஆதி [MASK] முதற்றே உலகு
Predictions:
Position 0: அகர
Position 1: பொருள்
Position 2: எழுத்த
Position 3: ##ெல்லாம்
Position 4: ஆதி
Position 5: என்று
Position 6: முதற்
Position 7: ##ற
Position 8: ##ே
Position 9: உலகு


In [31]:
# torch.cuda.empty_cache()

In [21]:
# save_directory = "../models/diamond_merged_tokenized"
# save_model(model, tokenizer, save_directory)

Model and vocabulary saved to ../models/valid_d_merged


In [29]:

def load_model(load_directory,model):
    model_path = os.path.join(load_directory, "bert_model11th_epoch.pth")
#     bert = BERT(vocab_size=vocab_size, d_model=d_model, n_layers=n_layers, heads=heads)
#     model = BERTLM(bert, vocab_size=vocab_size)
    print(f"Model and vocabulary loaded from {load_directory}")
    model.load_state_dict(torch.load(model_path))
    return model
#     vocab_path = os.path.join(load_directory, "vocab.json")
#     with open(vocab_path, 'r', encoding='utf-8') as f:
#         vocab = json.load(f)
    
#     tokenizer = Tokenizer(models.WordLevel(vocab=vocab, unk_token="[UNK]"))
#     tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
    
#     special_tokens = ["[PAD]","[MASK]", "[UNK]"]
#     tokenizer.add_special_tokens(special_tokens)

#     tokenizer.post_processor = processors.TemplateProcessing(
#         single="[CLS] $A [SEP]",
#         pair="[CLS] $A [SEP] $B:1 [SEP]:1",
#         special_tokens=[
#             ("[CLS]", tokenizer.token_to_id("[CLS]")),
#             ("[SEP]", tokenizer.token_to_id("[SEP]")),
#         ],
#     )
    
   
#     return model, tokenizer
model=load_model("/kaggle/input/bert/pytorch/default/1", model)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# loaded_model.to(device)

# sentence = "கற்றதனால் ஆய எல்லா பயன் என்கொல் நூல்களையும் கற்றவர்க்கு அக்கல்வி அறிவினாலாய பயன் யாது"
# word = "என்கொல்"
# similar_words = find_contextual_similar_words(sentence, word, tokenizer, model, device)
# print(f"Words similar to '{word}' in the context of '{sentence}': {similar_words}")

# masked_sentence = "கற்றதனால் [MASK] பயன் என்கொல் "
# predicted_word, predicted_sentence = predict_masked_word(masked_sentence, tokenizer, model, device)
# print(f"Original masked sentence: {masked_sentence}")
# print(f"Predicted word: {predicted_word}")
# print(f"Predicted sentence: {predicted_sentence}")


Model and vocabulary loaded from /kaggle/input/bert/pytorch/default/1


/tmp/ipykernel_36/86503471.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


In [11]:

def find_contextual_similar_words(sentence, word, tokenizer, model, device, top_n=5):
    model.eval()
    with torch.no_grad():
        tokens = [tokenizer.token_to_id(token) for token in sentence.split()]
        word_id = tokenizer.token_to_id(word)
        if word_id is None or word_id not in tokens:
            return []
        word_position = tokens.index(word_id)
        print(tokens)
        input_ids = torch.tensor([tokens]).to(device)
        attention_mask = torch.ones_like(input_ids).to(device)
        outputs = model.bert(input_ids, attention_mask)
        contextual_embedding = outputs[0, word_position]
        all_embeddings = model.bert.embedding.token.weight
        cosine_similarities = F.cosine_similarity(contextual_embedding, all_embeddings)
        top_indices = torch.argsort(cosine_similarities, descending=True)[1:top_n+1]
        similar_words = [tokenizer.id_to_token(idx.item()) for idx in top_indices]
        
        return similar_words

sentence = "வானூர்தி நிலையங்கள் உள்ளன"
word = "வானூர்தி"
similar_words = find_contextual_similar_words(sentence, word, tokenizer, model, device)
print(f"Words similar to '{word}' in the context of '{sentence}': {similar_words}") 

[7408, 11397, 2769]
Words similar to 'வானூர்தி' in the context of 'வானூர்தி நிலையங்கள் உள்ளன': ['1995', 'rep', 'தேவைப்படுகின்றன', '##ோயிக்', 'ஒன்றனைச்']


***claude code***

In [100]:
with torch.no_grad():
        word_id = tokenizer.token_to_id("விரிநீர்")
        if word_id is None:
            print('None')
        word_embedding = model.bert.embedding.token.weight[word_id]
        cosine_similarities = F.cosine_similarity(word_embedding, model.bert.embedding.token.weight)
        top_indices = torch.argsort(cosine_similarities, descending=True)[1:5]
        print([tokenizer.id_to_token(idx.item()) for idx in top_indices])

['நன்றல்லது', 'உயிருடையார்', 'ஒழுக்கத்தை', 'சாலும்']


In [ ]:
# def print_vocabulary(tokenizer):
#     vocab = tokenizer.get_vocab()
#     sorted_vocab=sorted(vocab.items(), key=lambda x: x[1])
#     print("Vocabulary:")
#     for word, index in sorted_vocab:
#         print(word,end=' ')
#     print(f"Total vocabulary size: {len(vocab)}")
